In [1]:
import numpy as np 
import matplotlib as mp 
import matplotlib.pyplot as plt 
import cython as c 
from Cython.Build import cythonize
%config InlineBackend.figure_format = 'pdf'
%load_ext Cython

Attempt to plot: 
$$\sum_{i=1}^{n=\infty} \frac{\cos(3^i \pi x)}{2^i}$$

In [2]:
%%time
%%cython
import numpy as np
cimport numpy as np
from libc.math cimport cos

def func(X, int n):
    cdef int i
    i = 0
    A = np.zeros(np.shape(X))
    while i <= n:
        B = np.cos(3**i * np.pi * X) / (2**i)
        A += B
        i += 1
    return A

cdef np.ndarray x
x = np.linspace(0, np.pi/3, 10**5)

F = func(x, 63)
print(F)

[ 2.          1.99996953  1.99993682 ... -1.46454196 -1.46454893
 -1.46455268]
CPU times: user 3min 21s, sys: 1min 14s, total: 4min 36s
Wall time: 2min 52s


In [3]:
x = np.linspace(0, np.pi/3, 10**5)
plt.figure(figsize=(16, 6))
plt.plot(x, F, linewidth=0.01)
plt.grid()
plt.show()

<Figure size 1152x432 with 1 Axes>

In [6]:
i = 64
x = np.linspace (0, np.pi/3, 10**4)
B = np.cos(3**i * np.pi * x) / (2**i)
B

array([5.421010862427522e-20, -2.2522563673930772e-20,
       -3.549530109780172e-20, ..., 6.6894591276975334e-21,
       -2.905695469752747e-20, -4.064081494766864e-20], dtype=object)